In [7]:
from hdmf.common import ExternalResources
from hdmf import Data
import pandas as pd

er = ExternalResources(name='example')

/Users/mavaylon/Research/hdmf_testing/hdmf/src/hdmf/container.py:181: UserWarning: ExternalResources is experimental -- it may be removed in the future and is not guaranteed to maintain backward compatibility
  warn(_exp_warn_msg(cls))


In [8]:
data = Data(name="species", data=['Homo sapien', 'Mus Musculus'])
data2 = Data(name="species", data=['Homo sapien', 'Mus Musculus'])
er.add_ref(container=data, field='', key='Homo sapien', resource_name='NCBI_Taxonomy',
           resource_uri='https://www.ncbi.nlm.nih.gov/taxonomy', entity_id='NCBI:txid9606',
           entity_uri='https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=9606')
er.add_ref(container=data, field='', key='Homo sapien', resource_name='NCBI_Taxonomy',
           resource_uri='https://www.ncbi.nlm.nih.gov/taxonomy', entity_id='NCBI:txid9606',
           entity_uri='https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=9606')
er.add_ref(container=data2, field='', key='Homo sapien', resource_name='NCBI_Taxonomy',
           resource_uri='https://www.ncbi.nlm.nih.gov/taxonomy', entity_id='NCBI:txid9606',
           entity_uri='https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=9606')
er.add_ref(container=data, field='', key='Mus Musculus', resource_name='NCBI_Taxonomy',
           resource_uri='https://www.ncbi.nlm.nih.gov/taxonomy', entity_id='NCBI:txid10090',
           entity_uri='https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=10090')

In [9]:
er.get_keys()

,key_name,resources_idx,entity_id,entity_uri
0,Homo sapien,0,NCBI:txid9606,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/...
1,Homo sapien,1,NCBI:txid9606,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/...
2,Homo sapien,2,NCBI:txid9606,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/...
3,Mus Musculus,3,NCBI:txid10090,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/...


In [10]:
er.keys.to_dataframe()

,key
0,Homo sapien
1,Mus Musculus


In [13]:
er.get_key('Homo sapien',container=data)

In [14]:
er.get_key('Homo sapien',container=data2)